In [922]:
import pandas as pd

In [923]:
df = pd.read_json('events.jsonl', lines = True)
df['event_type'].unique()
l = []
max_id = df['event_id'].max()
duplicates = df[df.duplicated(subset='event_id', keep=False)]
for id, x in duplicates.iterrows():
    l.append(id)
to_update = l[1:: 2]

for i, idx in enumerate(to_update):
    df.loc[idx, 'event_id'] = max_id + (i + 1)

In [924]:
df_register = df[df['event_type'] == 'registration']
df_session = df[df['event_type'] == 'session_ping']
df_match = df[df['event_type'] == 'match']

In [925]:
event_data = pd.DataFrame(list(df['event_data'].values))

In [926]:
def parse_event_info(df):
    unified_data  = []
    #print(pd.DataFrame(list(df['event_data'].values)))
    for event_id, event in zip(df['event_id'].values, df['event_data'].values):
        unified_event = {'event_id': event_id, **event}
        unified_data.append(unified_event)
    df = df.drop('event_data', axis = 1)
    #df = df.drop(['event_data', 'event_type'], axis = 1)
    df_event_data = pd.DataFrame(unified_data)

    return df, df_event_data

In [927]:
df_match, match_data = parse_event_info(df_match)
df_session, session_data = parse_event_info(df_session)
df_register, register_data = parse_event_info(df_register)

### Match data

In [928]:
match_data.describe()

,event_id,home_goals_scored,away_goals_scored
count,1878.000000,939.000000,939.000000
mean,24506.625666,1.538871,1.570820
std,11683.277419,1.383474,1.307694
min,402.000000,0.000000,0.000000
25%,15239.250000,1.000000,1.000000
50%,25263.500000,1.000000,1.000000
75%,34559.250000,2.000000,2.000000
max,42491.000000,5.000000,5.000000


In [929]:
hgs = match_data.dropna()['home_goals_scored'].apply(lambda x: x == int(x)).all()
ags = match_data.dropna()['away_goals_scored'].apply(lambda x: x == int(x)).all()
hgs, ags

(True, True)

### Register data

In [930]:
allowed_country = ['US', 'IT', 'JP', 'DE']
allowed_os = ['Android', 'Web', 'iOS']
register_data = register_data[register_data['country'].isin(allowed_country)]
register_data = register_data[register_data['device_os'].isin(allowed_os)]
register_data = register_data.drop_duplicates()
register_data['country'].unique()

array(['US', 'IT', 'JP', 'DE'], dtype=object)

In [931]:
register_data['device_os'].unique()

array(['Android', 'Web', 'iOS'], dtype=object)

In [932]:
register_data

,event_id,country,user_id,device_os
0,0,US,c393fd0e-1cc6-2be5-7836-46bf0324aac3,Android
1,1,IT,c33f4584-b23b-c1d8-493c-d01609de8895,Web
2,2,JP,9af49740-96f5-b0ee-5e25-02420ae59635,Android
3,3,IT,eeee3183-69ca-47e7-5826-00e9111f4efd,Android
4,4,DE,ec7d4222-6f41-2481-4fde-580f122088a5,Android
...,...,...,...,...
1397,42262,IT,1d26e398-aa58-96c5-3c8d-d523ed7aca64,Android
1398,42533,IT,6bc1028b-6830-b0f9-d958-83c8f8844056,Android
1399,42534,IT,e4d6b857-01a9-15fb-f4ca-de49fad0f7eb,Web
1400,42535,US,d76f5639-3918-0936-d2b0-2770fe3ab73a,iOS


### Session data

In [933]:
session_data = session_data.drop('type', axis = 1)
session_data

,event_id,user_id
0,88,75bd5125-db54-e35f-d802-db897f041728
1,89,75bd5125-db54-e35f-d802-db897f041728
2,90,75bd5125-db54-e35f-d802-db897f041728
3,91,75bd5125-db54-e35f-d802-db897f041728
4,92,db5b5fab-8f4d-3e27-dda1-494c73cf256d
...,...,...
39254,42528,00947bf5-63cf-82b2-5d14-d7aaa09c6b09
39255,42529,a9a3bec8-78cc-6246-b4a3-d5ef9412a806
39256,42530,00947bf5-63cf-82b2-5d14-d7aaa09c6b09
39257,42531,00947bf5-63cf-82b2-5d14-d7aaa09c6b09


In [934]:
df_session

,event_id,event_type,event_timestamp
88,88,session_ping,1728325170
89,89,session_ping,1728325230
90,90,session_ping,1728325290
91,91,session_ping,1728325350
92,92,session_ping,1728325858
...,...,...,...
42530,42528,session_ping,1730669986
42531,42529,session_ping,1730670007
42532,42530,session_ping,1730670046
42533,42531,session_ping,1730670106


In [935]:
df

,event_id,event_type,event_timestamp,event_data
0,0,registration,1728253060,"{'country': 'US', 'user_id': 'c393fd0e-1cc6-2b..."
1,1,registration,1728255330,"{'country': 'IT', 'user_id': 'c33f4584-b23b-c1..."
2,2,registration,1728255677,"{'country': 'JP', 'user_id': '9af49740-96f5-b0..."
3,3,registration,1728257778,"{'country': 'IT', 'user_id': 'eeee3183-69ca-47..."
4,4,registration,1728258117,"{'country': 'DE', 'user_id': 'ec7d4222-6f41-24..."
...,...,...,...,...
42534,42532,session_ping,1730670166,{'user_id': '00947bf5-63cf-82b2-5d14-d7aaa09c6...
42535,42533,registration,1730672274,"{'country': 'IT', 'user_id': '6bc1028b-6830-b0..."
42536,42534,registration,1730673050,"{'country': 'IT', 'user_id': 'e4d6b857-01a9-15..."
42537,42535,registration,1730674202,"{'country': 'US', 'user_id': 'd76f5639-3918-09..."


In [936]:
duplicates = df[df.duplicated(subset='event_timestamp', keep=False)]
duplicates

,event_id,event_type,event_timestamp,event_data
692,692,session_ping,1728516275,{'user_id': 'ef70b4c0-1773-44a3-9b95-f239ae97d...
693,693,session_ping,1728516275,{'user_id': '21636369-8b52-9b4a-97b7-50923ceb3...
698,698,session_ping,1728516335,{'user_id': '21636369-8b52-9b4a-97b7-50923ceb3...
699,699,session_ping,1728516335,{'user_id': 'ef70b4c0-1773-44a3-9b95-f239ae97d...
704,704,session_ping,1728516395,{'user_id': '21636369-8b52-9b4a-97b7-50923ceb3...
...,...,...,...,...
42418,42416,session_ping,1730669176,{'user_id': '8623121d-e0bb-f37a-9459-4d8b75673...
42434,42432,session_ping,1730669272,{'user_id': 'f853367e-666b-a363-86e6-4e7d9885a...
42435,42433,match,1730669272,{'match_id': '1b8fc36e-9123-11ef-b02f-bafd2d38...
42492,42490,session_ping,1730669566,{'user_id': '00947bf5-63cf-82b2-5d14-d7aaa09c6...


#### Checking if match and ping can happen at same time

In [937]:
test, teest = parse_event_info(df)

In [938]:
kekw = teest.merge(test)

In [939]:
kekw['user_id'] = kekw['user_id'].fillna(kekw['home_user_id'])
oof = kekw[['event_timestamp', 'user_id']]

In [940]:
duplicates = kekw[kekw.duplicated(subset=['event_timestamp', 'user_id'], keep=False)]
ooga = duplicates[['user_id', 'event_timestamp', 'event_type']]
ooga

,user_id,event_timestamp,event_type
1706,5bc8fbbc-bde5-c099-4164-d8399f767c45,1728689838,session_ping
1707,5bc8fbbc-bde5-c099-4164-d8399f767c45,1728689838,match
2568,a8d42934-33e7-98a0-e81f-9b0cbf4e7af6,1728862853,session_ping
2569,a8d42934-33e7-98a0-e81f-9b0cbf4e7af6,1728862853,match
3220,9530fcd9-d6fd-1d9b-6203-2801b65c1c28,1728950476,session_ping
...,...,...,...
41303,45cc9e19-9a2e-ec11-9150-fe7ee544f243,1730665567,match
41856,2b702f67-99c7-414a-5c94-687cdd35fde9,1730667142,session_ping
41857,2b702f67-99c7-414a-5c94-687cdd35fde9,1730667142,match
42144,736d5f3e-d253-45d0-91bd-35fe3bc33c99,1730668179,session_ping


: 

#### All playtime is registered in sessions_info